# Prepocessing - Numerical Data

Très souvent besoinde rescale les données en entré car la plupart des algorithmes de machine learning présuppose que toutes les features sont à la même échelle. 

# Import

In [89]:
import  numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import Binarizer
from sklearn.impute import SimpleImputer

# Sommaire

[I. Rescaling, Standardizing, Normalizing features](#Partie1)

[II. Generating polynomial and interaction features](#Partie2)

[III. Detection and Handling of outliers](#Partie3)

[IV. Discretizating features](#Partie4)

[V. Deleting and imputing missing values](#Partie5)


## I. Rescaling, Standardizing, Normalizing features <a class="anchor" id="Partie1"></a>

##### StandardScaler : 

On approxime les features de sorte à ce qu'elles suivent une distribution normal centrée réduite.
$$\begin{equation}
x_{i}^{\prime}=\frac{x_{i}-x_{moy}}{\sigma}
\end{equation} $$

In [90]:
feature=np.array([[-500],[-200],[0],[400],[900]]) #create feature

In [91]:
standard_scale=preprocessing.StandardScaler()
scaled_feature=standard_scale.fit_transform(feature)
scaled_feature

array([[-1.27194541],
       [-0.65648795],
       [-0.24618298],
       [ 0.57442696],
       [ 1.60018938]])

##### MinMaxScaler : 

utilise le minimum et le maximum du vecteur d'entrée pour réechelonner dans une range précisé (0,1).
$$\begin{equation}
x_{i}^{\prime}=\frac{x_{i}-\min (x)}{\max (x)-\min (x)}
\end{equation} $$

In [92]:
feature=np.array([[-500],[-200],[0],[400],[900]]) #create feature

In [93]:
minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
scaled_feature=minmax_scale.fit_transform(feature)
scaled_feature

array([[0.        ],
       [0.21428571],
       [0.35714286],
       [0.64285714],
       [1.        ]])

La distinction entre minmax ou standardisation se fait souvent dépendamment de l'algorithme utilisé, très souvent standardiser sera préféré pour de la PCA par exemple mais minmax peut être recommandé pour des réseaux de neurones.

##### Normalizer : 

Ici on souhaite normalisé (une somme des normes égale à 1) les valeurs des observations (ie plutôt qu'agir directement sur tout le jeu de donnée comme min-max ou standard on va normalisé features par features) avec la norme que l'on souhaite utiliser

In [94]:
feature=np.array([[0.5,0.5],[-200,456],[0,7],[56,400]]) #create feature

In [95]:
normalizer=Normalizer(norm="l2")
scaled_feature=normalizer.transform(feature)
scaled_feature

array([[ 0.70710678,  0.70710678],
       [-0.4016615 ,  0.91578821],
       [ 0.        ,  1.        ],
       [ 0.13864784,  0.99034175]])

## II. Generating polynomial and interaction features <a class="anchor" id="Partie2"></a>

On vient ajouter les termes du polynômes du degré mentionné (exemple avec degré=2), on vient ajouter
$$\begin{equation}
x_{1}^{2} , \space x_{1}*x_{2} , \space x_{2}^{2}
\end{equation} $$

On peut également choisir de ne regarder que les termes au puissance ou que les termes d'interaction ou les 2.
Cette démarche est utile s'il existe et si l'on souhaite ajouter des relations non linéaires entre les features et la valeur à prédire. Par s'il existe un effet non constant entre l'interaction de 2 variables et la valeur à prédire (ex: l'age et une condition médical on peut par exmeple donner comme variable l'age au carré ou cube pour montrer l'impact plus  important de l'age sur une condition médical pour les valeurs hautes)

In [96]:
feature=np.array([[-2,6],[1,7]]) #create feature

In [97]:
polynomial_interaction=PolynomialFeatures(degree=2, include_bias=False)
feature_interaction=polynomial_interaction.fit_transform(feature)
feature_interaction

array([[ -2.,   6.,   4., -12.,  36.],
       [  1.,   7.,   1.,   7.,  49.]])

## III. Detection and Handling of outliers <a class="anchor" id="Partie3"></a>

##### Détecter les outliers: EllipticEnvelope : 

Une solution pour la détection d'outlier est de supposer que les données sont ditribuees suivant une loi normale et de "dessiner" une ellipse autour de ces données et classifier toutes les données dans l'ellipse comme des "inlier" (1) et celle en dehors comme "outlier" (-1)

La limitation ici est qu'il faut préciser une valeur de "contamination" qui est la proportion a priori d'outlier dans le jeu de donnée (valeur non connue en général)

In [98]:
outlier_detector=EllipticEnvelope(contamination=0.1)
outlier_detector.fit(feature)
outlier_detector.predict(feature)

array([-1, -1])

##### Détecter les outliers: IsolationForest : 

cf TP 3 Machine learning detection anomalie

##### Gestion des outliers

- Première stratégie: dropper les outliers directement du jeu de donnée
- Deuxième stratégie: on définit une nouvelle features "Outlier" qui donne un booleen (0 ou 1) si l'observation est une outleir ou pas et on peut directement utiliser cette feature dans le set de donnée
- Troisième stratégie: On transforme la feature avec outlier pour ne plus avoir les effets d'anomalie (via standardisation, passage au log etc)

Pour savoir quelle strat utilisée cela dépend directement du "pourquoi ces observations sont des outliers" :
- 1: Est ce qu'elles sont purement issu d'erreur de mesure ? alors on peut juste les dropper ou les remplacer
- 2: Si elles sont des valeurs extrêmes réelles du jeu de donnée alors les marquer et ajouter une feature "outlier" est plus approprié

De la même manière cela dépend aussi du but de l'algorithme de ML, il est raisonnable de penser que pour ceratines problématiques les valeurs extrêmes du jeu de données ne sont pas driver par les même facteurs que la vaste majorité des données et dépendent de facteur absent du jeu de donnée.

#### RobustScaler: 
Si présence d'outlier non négligeable dans un jeu de donnée on pêut utilier pour le preprocessing des données:

## IV. Discretizating features <a class="anchor" id="Partie4"></a>

#### Binarizer: 

Si on a des valeurs numériques mais que l'on souhaite les discrétiser pour une problématique données (ex: pic de pollution avec un seuil de concentration de gaz au dessus duquel 1 sinon 0)

In [99]:
feature=np.array([[6],[20],[12],[56],[78]]) #create feature

In [100]:
binarizer=Binarizer(threshold=18)
binarized_feature=binarizer.fit_transform(feature)
binarized_feature

array([[0],
       [1],
       [0],
       [1],
       [1]])

#### Digitize: 

Si on a des valeurs numériques mais que l'on souhaite les discrétiser pour une problématique données, ici on définit des tranches qui prendront 0,1,2 etc comme valeur

In [101]:
digitized_feature=np.digitize(feature,bins=[20,30,60])
digitized_feature

array([[0],
       [1],
       [0],
       [2],
       [3]], dtype=int64)

## V. Handling missing values <a class="anchor" id="Partie5"></a>

#### Suppression des valeurs manquantes

Premier cas on peut simplement supprimer les valeurs manquantes avec df.dropna() mais cela ne devrait pas être priviligié comme solution car l'algo perd alors en information et peut introduire du biais dans l'algo. On peut définir 3 types de valeurs manquantes: 

- 1 :(MCAR: missing completely at random): La proba qu'une valeur manque est completement indépendante de quoi que ce soit.
- 2 :(MAR: missing at random): La proba n'est pas completement indépendante mais dépend d'une info dans une autre feature. Dans un survey la non réponse de l'interrogé peut être une information en elle même (pas envie de répondre, pas la possiblité de répondre etc)
- 3 :(MNAR: missing not at random): La proba n'est pas indépendante mais dépend d'information non contenue dans les features

Dans les cas 1 et 2 il peut être acceptable de supprimer les valeurs manquantes mais plus délicat dans le 3eme cas car injecte automatiquement du biais.

#### Imputing valeurs manquantes

- Première solution: utiliser KNN si petit volume de donnée est très efficace
- Deuxième solution: Imputer de scikit avec la moyenne (penser à ajouter une feature avec 0 ou 1 selon si une valeur était manquante)

In [103]:
mean_imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
feature_mean_imputed=mean_imputer.fit_transform(feature)